In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
df = pd.read_csv('smartcart_customers.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

data preprocessing

`handle missing value`

In [ ]:
df['Income'] = df['Income'].fillna(df['Income'].mean())

In [ ]:
df.head()

`feature engineering`

In [ ]:
# year birth to age 

In [ ]:
df['Age'] = 2026 - df['Year_Birth']

In [ ]:
df.head()

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)

refrence_date = df['Dt_Customer'].max()

In [ ]:
df['Customer_tanuar_days'] = (refrence_date - df['Dt_Customer']).dt.days

In [ ]:
# spending

In [ ]:
#add total spending

In [ ]:
df.columns

In [ ]:
# Method 1: List of columns and sum
spending_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_spending'] = df[spending_cols].sum(axis=1)

In [ ]:
df.columns

In [ ]:
total_chidren = ['Kidhome','Teenhome']

df['total_childern'] = df[total_chidren].sum(axis = 1)

In [ ]:
# education

df['Education'].value_counts()

df['Education'] = df['Education'].replace({
    'Basic': 'undergraduate','2n Cycle':'undergraduate',
    'Graduation':'graduate',
    'Master':'postgraduate', 'PhD':'postgraduate'
})


In [ ]:
df['Education'].value_counts()

In [ ]:
# marital_status

df['Marital_Status'].value_counts()

In [ ]:
df['Living_with'] = df['Marital_Status'].replace({
    "Married":'Patner', "Together":'Patner',
    "Divorced":'Alone', "Single":'Alone', "Widow":'Alone',"Absurd":'Alone',"YOLO":'Alone'
    
})

In [ ]:
df['Living_with'].value_counts()

In [ ]:
# drop column

In [ ]:
df.columns

In [ ]:
cols = ['ID', 'Year_Birth', 'Marital_Status','Kidhome','Teenhome', 'Dt_Customer']

spending_col = ['MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds']


cols_to_drop = cols + spending_col

clean_df = df.drop(cols_to_drop, axis = 1)


In [ ]:
clean_df.shape

In [ ]:
clean_df.head()

`removing outllier`

In [ ]:
num_col = clean_df.select_dtypes(include=['float64', 'int64']).columns.tolist()


In [ ]:
num_col

In [ ]:
cat_col = clean_df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
cat_col

In [ ]:
clean_df.columns

In [ ]:
# pair plot : find visually outlier


cols = ['Income','Recency','Response','Age','Total_spending','total_childern']
sns.pairplot(clean_df[cols])
plt.show()

In [ ]:
# removing outlier

clean_df = clean_df[(clean_df['Age'] < 90)]
clean_df = clean_df[(clean_df['Income']) < 600000]

In [ ]:
len(clean_df)

In [ ]:
plt.figure(figsize=(8, 10))

corr = clean_df.corr(numeric_only=True)

sns.heatmap(
    corr, 
    annot=True,           # Show values in cells
    cmap="coolwarm",       # Color scheme
    fmt='.2f',            # Format numbers to 2 decimal places
    linewidths=1,         # Width of lines between cells
    linecolor='white',    # Color of lines between cells
    cbar=True,            # Show color bar
    square=True,          # Make cells square
    annot_kws={'size':10}, # Size of annotation text (this replaces your numeric_only attempt)
    vmin=-1,              # Minimum value for colormap
    vmax=1                # Maximum value for colormap
)
plt.title('Correlation Matrix', fontsize=16)
plt.tight_layout()
plt.show()

### `encoding`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cat_col

In [ ]:
ohe = OneHotEncoder()
cat_cols = ['Education', 'Living_with']
enc_col = ohe.fit_transform(clean_df[cat_cols])

# Create DataFrame with encoded columns - corrected method name
encoded_df = pd.DataFrame(
    enc_col.toarray(), 
    columns=ohe.get_feature_names_out(cat_cols),  # Note: 'feature_names_out' (with 's')
    index=clean_df.index
)

# Display the result
encoded_df.head()

In [ ]:
encoded_df= pd.concat([clean_df.drop(columns = cat_cols),encoded_df], axis = 1)

In [ ]:
encoded_df

### scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = encoded_df

In [ ]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled

### visulaizing

In [ ]:
#2d

from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

X_pca = pca.fit_transform(X_scaled)

plt.scatter(X_pca[:, 0], X_pca[:, 1])  # Note the comma, not colon
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA Results')
plt.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame for plotly
plot_df = pd.DataFrame(
    X_pca, 
    columns=['PC1', 'PC2', 'PC3']
)

# Add any categorical columns for coloring
if 'target_column' in clean_df.columns:
    plot_df['Target'] = clean_df['target_column']

# Create interactive 3D plot
fig = px.scatter_3d(
    plot_df, 
    x='PC1', y='PC2', z='PC3',
    color='Target' if 'target_column' in clean_df.columns else None,
    title='Interactive 3D PCA Visualization',
    labels={'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.2%})',
            'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.2%})',
            'PC3': f'PC3 ({pca.explained_variance_ratio_[2]:.2%})'}
)

fig.update_traces(marker=dict(size=5, line=dict(width=1, color='black')))
fig.show()

In [ ]:
!pip install kneed
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator
wcss = []

for k in range(1,21):
    kmeans = KMeans(n_clusters = k, random_state = 42)
    kmeans.fit_predict(X_pca)
    wcss.append(kmeans.inertia_)

In [ ]:
kl = KneeLocator(
    x=range(1,21),  # or just k_range if it's already a list/array
    y=wcss, 
    curve="convex", 
    direction="decreasing"
)

# Get the optimal k
optimal_k = kl.elbow
print(f"Optimal number of clusters: {optimal_k}")

In [ ]:
plt.plot(range(1,21), wcss, marker='o')
plt.xlabel("k value")
plt.ylabel("wcss")

### Silhoutee Score

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
scores = []
for k in range(2,21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_pca)
    score = silhouette_score(X_pca ,labels)
    scores.append(score)

In [ ]:
plt.plot(range(2,21),scores , marker='o')
plt.xlabel("k")
plt.ylabel("Silhoutee score")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create KMeans with correct parameter name
kmeans = KMeans(n_clusters=4, random_state=42)
labels_kmeans = kmeans.fit_predict(X_pca)  # store labels

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with the cluster labels
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], 
                     c=labels_kmeans, cmap='viridis', s=50, alpha=0.7)

# Add color bar and labels
plt.colorbar(scatter, ax=ax, label='Cluster')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('K-Means Clustering on PCA-Reduced Data (k=4)')

plt.show()

In [ ]:
# agglomerative

from sklearn.cluster import AgglomerativeClustering

In [ ]:
agg_clf = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels_agg = agg_clf.fit_predict(X_pca)



fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with the cluster labels
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], 
                     c=labels_agg, cmap='viridis', s=50, alpha=0.7)

# Add color bar and labels
plt.colorbar(scatter, ax=ax, label='Cluster')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('K-Means Clustering on PCA-Reduced Data (k=4)')

plt.show()

### characterusaion

In [ ]:
X['labels'] = labels_agg

In [ ]:
X

In [ ]:
pal = ['red','blue','yellow','green']
sns.countplot(x=X['labels'],palette = pal, hue = X['labels'])

In [ ]:
X.columns

In [ ]:
sns.scatterplot(x = X['Income'], y= X['Total_spending'], hue = X['labels'], palette = pal)

In [ ]:
# cluster_summeries

cluster_summery = X.groupby("labels").mean()
print(cluster_summery)